In [1]:
import csv
import pandas as pd
import numpy as np
import os
from collections import OrderedDict

In [2]:
listasPath = []
for i in range(2):
    listasPath.append(raw_input("Ingresar ruta a lista %s: " % (i+1)))
    if not(os.path.isfile(listasPath[i])):
        print "Path a Archivo no es válido"
        exit

Ingresar ruta a lista 1: /home/juan/Documentos/UNLu/taller_libre_1/2016/practica/tp3/punto5/retrieval/tf_idf/TF_IDF_0.res
Ingresar ruta a lista 2: /home/juan/Documentos/UNLu/taller_libre_1/2016/practica/tp3/punto5/retrieval/bm25/BM25b0.75_1.res


In [3]:
lista = []
for i in range(2):
    lista.append(pd.read_csv(listasPath[i], sep=" ", header = None, names=["Id query", "Q", "Id doc", "Ranking", "Similtud", "Modelo"], index_col=["Id query","Id doc"]))
    modelo = lista[i]['Modelo'].iloc[0]
    lista[i].drop(['Q','Modelo', "Similtud"], axis=1, inplace=True)
    lista[i].columns = ["Ranking - "+modelo]
lista[1].tail()

Ranking - BM25b0.75
Id query Id doc                     
4        111                      46
         256                      47
         54                       48
5        374                       0
         308                       1

In [4]:
def sp(a):
    CONSTANTE = 6.0 #  valor que se multiplica por sumatoria del cuadrado de las diferencias
    N = len(a) - 1.0 # posicion maxima del ranking
    
    diferencia = a[a.columns[2]] - a[a.columns[3]]
    diferenciaCuadrado = np.power(diferencia, 2.0)
    sumDifCuad = sum(diferenciaCuadrado)
    dividendo = sumDifCuad * CONSTANTE
    divisor = N * (np.power(N, 2.0) - 1.0)

    if divisor == 0 or dividendo == 0:
        cs = 1
    else:
        cs = 1 - (dividendo / divisor) 
    
    return cs


In [5]:
def getTopN(df, n):
    return df.sort_values(["Id query",df.columns[0]]).groupby("Id query").head(n)

In [6]:
tops = {
    10: [],
    25: [],
    50: []
}
for k in tops:
    # Obtengo top k para ambas listas
    ranks = [getTopN(df.reset_index(), k).set_index(["Id query", "Id doc"]) for df in lista]
    
    # Concateno listas
    ranksConcat = pd.concat(ranks, axis=1, join="outer")

    # Docs id que se encuentran en una lista y no en otra se rellenan con el rank maximo + 1
    fillValues = ranksConcat.max(level="Id query") + 1
    ranksConcat = ranksConcat.fillna(value=fillValues)

    # Calculo coeficiente de spearman
    tops[k] = ranksConcat.reset_index().groupby(["Id query"]).apply(sp)
    
    # Muestro resultados
    print "\ntop ", k
    print tops[k]


top  25
Id query
1    1.000000
2    0.377340
3    0.843419
4    1.000000
5    1.000000
dtype: float64

top  10
Id query
1    1.000000
2    0.151648
3    0.733333
4    1.000000
5    1.000000
dtype: float64

top  50
Id query
1    1.000000
2    0.746697
3    0.913482
4    1.000000
5    1.000000
dtype: float64


In [7]:
cs = pd.concat([tops[k] for k in sorted(tops)], axis=1,join="outer")
cs.columns = ["Top %d" % k for k in sorted(tops)]
cs

,top10,top25,top50
Id query,,,
1,1.000000,1.000000,1.000000
2,0.151648,0.377340,0.746697
3,0.733333,0.843419,0.913482
4,1.000000,1.000000,1.000000
5,1.000000,1.000000,1.000000


In [8]:
cs.to_csv("coeficiente_spearman.csv")